# Summary

This notebook analyzes yiqtol forms in the 3rd person. 

## Selection

Verbs from `data.json` are if tagged as `DEONTIC`. They are also filtered based on the feature `ps` in BHSA (condition `ps=p3`).

# [Initialization]

In [29]:
# Load data
with open("data.json", 'r') as f:
    data = json.load(f)
# Convert JSON keys to integers.
data = { int(k): v for k, v in data.items() }

In [30]:
# whose wish/desire is relevant?
SPK_WISH = "expresses wish of speaker"
SUBJ_WISH = "expresses or assumes wish of subject (who is not speaker)"
ADDR_WISH = "expresses or assumes wish of addressee (who is not subject)"
MORAL = "expresses moral obligation rather than will"

# agency refers to who is in control or who is exerting their will through what is said
SPK_AGENCY = "expresses agency of speaker"
ADDR_AGENCY = "expresses/appeals to agency of addressee"
NO_AGENCY = "does not express/appeal to agency of speaker or addressee"

# authority here refers to the social setting regardless of speech content
HIGHER = "speaker has higher authority than addressee"
LOWER = "speaker has lower authority than addressee"
EQUAL = "speaker has roughly equal authority as addressee"
A_UNCLEAR = "authority is unclear or ambivalent"

DEONTIC = "certainly or probably deontic"
D_QUESTIONABLE = "questionable whether verb expresses deontic modality"

SELECTION = [DEONTIC]

In [38]:
# morphology
LONG = "long yiqtol"
SHORT = "short yiqtol"
LONG_SHORT = "long or short yiqtol"
LONG_N = "paragogic nun"

# add a tag to group paragogic-nun forms with other long yiqtols
# (this is for convenience in this notebook only; data is not saved at the end of the notebook)
ANY_LONG = "long yiqtol or yiqtol with paragogic nun"

for tags in data.values():
    if LONG in tags or LONG_N in tags:
        tags.append(ANY_LONG)

In [32]:
# position in clause
INIT = "clause-initial (1st word in clause)"
NON_INIT = "not 1st word in clause"

In [33]:
import json
from IPython.display import display, Markdown
from tf.app import use
A = use('etcbc/bhsa')
F = A.TF.api.F
L = A.TF.api.L

This is Text-Fabric 9.3.2
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

122 features found and 0 ignored


In [34]:
AUTHORITY = [HIGHER, LOWER, EQUAL, A_UNCLEAR]
POSITION = [INIT, NON_INIT]

def matches_selection(tags):
    for tag in SELECTION:
        if tag not in tags:
            return False    
    if D_QUESTIONABLE in tags:
        return False
    return True

data_subset = {k: v for k, v in data.items() if matches_selection(v)}

def show_subset(tags=[], features="", group_by_position=True, group_by_authority=False):
    # dict (key: authority tag; value: list of verb nodes) 
    by_auth = {auth: [] for auth in AUTHORITY}
    by_pos = {pos: [] for pos in POSITION}

    # filter by specified features
    prelim_results = A.search(f'''
book book=Reges_I|Reges_II
    clause
        word vt=impf ps=p3 {features}
''')

    # filter by specified tags
    count = 0
    for bk, cl, word in prelim_results:
        if word in data_subset:
            for t in tags:
                if t not in data_subset[word]:
                    continue
            
                # extract word's tag for authority
                auth = [tag for tag in data[word] if tag in AUTHORITY][0]
                
                # store clause & word
                by_auth[auth].append((cl, word))

                # extract word's tag for position
                pos = [tag for tag in data[word] if tag in POSITION][0]
                
                # store clause & word
                by_pos[pos].append((cl, word))

                count += 1
    
    print(f"{count} occurrences total\n")
    for pos, nodes in by_pos.items():
        print(f"{len(nodes):3}x {pos}")
    print()
    for auth, nodes in by_auth.items():
        print(f"{len(nodes):3}x {auth}")
    
    if group_by_position:
        for pos, nodes in by_pos.items():
            if len(nodes) > 0:
                display(Markdown(f'### {pos}'))
                A.table(nodes)

    if group_by_authority:
        for auth, nodes in by_auth.items():
            if len(nodes) > 0:
                display(Markdown(f'### {auth}'))
                A.table(nodes)

    if not group_by_position and not group_by_authority:
        nodes = [tup for tup_list in by_auth.values() for tup in tup_list]
        nodes.sort()
        A.table(nodes)

# Long yiqtol (not including ambiguous forms)

In [37]:
show_subset(tags=[ANY_LONG])

  0.55s 345 results
15 occurrences total

  0x clause-initial (1st word in clause)
 15x not 1st word in clause

 11x speaker has higher authority than addressee
  2x speaker has lower authority than addressee
  0x speaker has roughly equal authority as addressee
  2x authority is unclear or ambivalent


### not 1st word in clause

n,p,clause,word
1,1_Kings 1:37,כֵּ֖ן יִֽהְיֶ֣ה עִם־שְׁלֹמֹ֑ה,יִֽהְיֶ֣ה
2,1_Kings 2:23,כֹּ֣ה יַֽעֲשֶׂה־לִּ֤י אֱלֹהִים֙,יַֽעֲשֶׂה־
3,1_Kings 2:23,וְכֹ֣ה יֹוסִ֔יף,יֹוסִ֔יף
4,1_Kings 2:33,וּלְדָוִ֡ד וּ֠לְזַרְעֹו וּלְבֵיתֹ֨ו וּלְכִסְאֹ֜ו יִהְיֶ֥ה שָׁלֹ֛ום עַד־עֹולָ֖ם מֵעִ֥ם יְהוָֽה׃,יִהְיֶ֥ה
5,1_Kings 2:45,וְכִסֵּ֣א דָוִ֗ד יִהְיֶ֥ה נָכֹ֛ון לִפְנֵ֥י יְהוָ֖ה עַד־עֹולָֽם׃,יִהְיֶ֥ה
6,1_Kings 3:26,לֹ֥א יִהְיֶ֖ה,יִהְיֶ֖ה
7,1_Kings 5:19,הֽוּא־יִבְנֶ֥ה הַבַּ֖יִת לִשְׁמִֽי׃,יִבְנֶ֥ה
8,1_Kings 19:2,כֹּֽה־יַעֲשׂ֤וּן אֱלֹהִים֙,יַעֲשׂ֤וּן
9,1_Kings 19:2,וְכֹ֣ה יֹוסִפ֔וּן,יֹוסִפ֔וּן
10,1_Kings 20:10,כֹּֽה־יַעֲשׂ֥וּן לִ֛י אֱלֹהִ֖ים,יַעֲשׂ֥וּן


# Short yiqtol (not including ambiguous forms)

In [36]:
show_subset(tags=[SHORT])

  0.70s 345 results
14 occurrences total

 14x clause-initial (1st word in clause)
  0x not 1st word in clause

  3x speaker has higher authority than addressee
  5x speaker has lower authority than addressee
  1x speaker has roughly equal authority as addressee
  5x authority is unclear or ambivalent


### clause-initial (1st word in clause)

n,p,clause,word
1,1_Kings 1:2,וּתְהִי־לֹ֖ו סֹכֶ֑נֶת,תְהִי־
2,1_Kings 1:25,יְחִ֖י הַמֶּ֥לֶךְ אֲדֹנִיָּֽהוּ׃,יְחִ֖י
3,1_Kings 1:31,יְחִ֗י אֲדֹנִ֛י הַמֶּ֥לֶךְ דָּוִ֖ד לְעֹלָֽם׃ פ,יְחִ֗י
4,1_Kings 1:34,יְחִ֖י הַמֶּ֥לֶךְ שְׁלֹמֹֽה׃,יְחִ֖י
5,1_Kings 1:39,יְחִ֖י הַמֶּ֥לֶךְ שְׁלֹמֹֽה׃,יְחִ֖י
6,1_Kings 1:47,יֵיטֵ֨ב אֱלֹהִ֜ים אֶת־שֵׁ֤ם שְׁלֹמֹה֙ מִשְּׁמֶ֔ךָ,יֵיטֵ֨ב
7,1_Kings 8:57,יְהִ֨י יְהוָ֤ה אֱלֹהֵ֨ינוּ֙ עִמָּ֔נוּ,יְהִ֨י
8,1_Kings 10:9,יְהִ֨י יְהוָ֤ה אֱלֹהֶ֨יךָ֙ בָּר֔וּךְ,יְהִ֨י
9,1_Kings 17:21,תָּ֥שָׁב נָ֛א נֶֽפֶשׁ־הַיֶּ֥לֶד הַזֶּ֖ה עַל־קִרְבֹּֽו׃,תָּ֥שָׁב
10,1_Kings 21:2,וִֽיהִי־לִ֣י לְגַן־יָרָ֗ק,יהִי־


# Long or short yiqtol (ambiguous forms)

In [39]:
show_subset(tags=[LONG_SHORT])

  0.54s 345 results
73 occurrences total

 50x clause-initial (1st word in clause)
 23x not 1st word in clause

 36x speaker has higher authority than addressee
 23x speaker has lower authority than addressee
  2x speaker has roughly equal authority as addressee
 12x authority is unclear or ambivalent


### clause-initial (1st word in clause)

n,p,clause,word
1,1_Kings 1:2,יְבַקְשׁ֞וּ לַאדֹנִ֤י הַמֶּ֨לֶךְ֙ נַעֲרָ֣ה בְתוּלָ֔ה,יְבַקְשׁ֞וּ
2,1_Kings 1:37,וִֽיגַדֵּל֙ אֶת־כִּסְאֹ֔ו מִ֨כִּסֵּ֔א אֲדֹנִ֖י הַמֶּ֥לֶךְ דָּוִֽד׃,יגַדֵּל֙
3,1_Kings 1:47,וִֽיגַדֵּ֥ל אֶת־כִּסְאֹ֖ו מִכִּסְאֶ֑ךָ,יגַדֵּ֥ל
4,1_Kings 1:51,יִשָּֽׁבַֽע־לִ֤י כַיֹּום֙ הַמֶּ֣לֶךְ שְׁלֹמֹ֔ה,יִשָּֽׁבַֽע־
5,1_Kings 2:17,וְיִתֶּן־לִ֛י אֶת־אֲבִישַׁ֥ג הַשּׁוּנַמִּ֖ית לְאִשָּֽׁה׃,יִתֶּן־
6,1_Kings 2:21,יֻתַּ֖ן אֶת־אֲבִישַׁ֣ג הַשֻּׁנַמִּ֑ית לַאֲדֹנִיָּ֥הוּ אָחִ֖יךָ לְאִשָּֽׁה׃,יֻתַּ֖ן
7,1_Kings 8:26,יֵאָ֤מֶן נָא֙ דְּבָ֣רְךָ֔,יֵאָ֤מֶן
8,1_Kings 8:57,אַל־יַעַזְבֵ֖נוּ,יַעַזְבֵ֖נוּ
9,1_Kings 8:57,וְאַֽל־יִטְּשֵֽׁנוּ׃,יִטְּשֵֽׁנוּ׃
10,1_Kings 8:59,וְיִֽהְי֨וּ דְבָרַ֜י אֵ֗לֶּה קְרֹבִ֛ים אֶל־יְהוָ֥ה אֱלֹהֵ֖ינוּ יֹומָ֣ם וָלָ֑יְלָה,יִֽהְי֨וּ


### not 1st word in clause

n,p,clause,word
1,1_Kings 1:35,וְה֥וּא יִמְלֹ֖ךְ תַּחְתָּ֑י,יִמְלֹ֖ךְ
2,1_Kings 1:36,כֵּ֚ן יֹאמַ֣ר יְהוָ֔ה אֱלֹהֵ֖י אֲדֹנִ֥י הַמֶּֽלֶךְ׃,יֹאמַ֣ר
3,1_Kings 8:36,אֲשֶׁ֣ר יֵֽלְכוּ־בָ֑הּ,יֵֽלְכוּ־
4,1_Kings 11:2,וְהֵם֙ לֹא־יָבֹ֣אוּ בָכֶ֔ם,יָבֹ֣אוּ
5,1_Kings 14:2,וְלֹ֣א יֵֽדְע֔וּ,יֵֽדְע֔וּ
6,1_Kings 18:23,וְאֵ֖שׁ לֹ֣א יָשִׂ֑ימוּ,יָשִׂ֑ימוּ
7,1_Kings 18:36,הַיֹּ֣ום יִוָּדַ֗ע,יִוָּדַ֗ע
8,1_Kings 18:40,אִ֛ישׁ אַל־יִמָּלֵ֥ט מֵהֶ֖ם,יִמָּלֵ֥ט
9,1_Kings 18:44,וְלֹ֥א יַעַצָרְכָ֖ה הַגָּֽשֶׁם׃,יַעַצָרְכָ֖ה
10,1_Kings 20:6,כָּל־מַחְמַ֣ד עֵינֶ֔יךָ יָשִׂ֥ימוּ בְיָדָ֖ם,יָשִׂ֥ימוּ
